**Unstructured Data Analytics: Group Assignment #1**

# Group Members:
1. Rithu Anand Krishnan
2. Manvi Mahajan
3. Paul Wen (Yilin Wen)
4. Alex Yu
5. Judy Chen (Yu-Ting Chen)

In [ ]:
## Imports
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
import matplotlib.pyplot as p
from statistics import mean
from sklearn.manifold import MDS
import sklearn.datasets as dt
import seaborn as sns
from nltk import pos_tag
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from sklearn.linear_model import LinearRegression
import os
import re
import nltk
import requests
import csv
import matplotlib
import matplotlib.pyplot as plt
import string
import math
nltk.download('all')

# Data

Read in data

In [ ]:
#Created dataframe from scraped tweets
df = pd.read_csv("6205 beer reviews of 250 beers.csv", names=["Product_name", "product_review", "user_ratings","overall_rating","clean_review"])

## Task E. 
Create an evaluation score for each beer that uses both similarity and sentiment scores. E.g.,
total score = average of (similarity score + sentiment score) or a multiplicative model.

# Task H
Using the top four attributes of beer (from word frequency analysis), calculate the lifts between
these attributes and any 10 beers in your data. Choose one beer, and find the most similar beer (among
the remaining 9) using the lift values. Explain your method.

In [ ]:
#takes out top 10 brands from tuple in sorted brand freq
brands = [ item[0] for item in sorted_brand_freq]

# finds lift between two entities
def lift(A, B, messages):
    A=A.lower()
    B=B.lower()
    A_count = 0
    B_count = 0
    AB_count = 0
    n=len(messages)
    for message in messages:
      message=message.lower()
      if A in message:
        A_count+=1
      if B in message:
        B_count+=1
      if A in message and B in message:
        AB_count+=1
    lift_val= (n*AB_count)/(A_count*B_count)
    if A == B:
      return 1
    else:
      return round(lift_val,4)

# utilizes lift function above and runs agaainst all the brands in the brand list. 
# Returns dataframe of lift values with useless values removed
def lift_everything(brands,messages):
  results=[]
  counter=0
  for b in brands:
    results.append([])
    for f in brands:
      k=lift(b,f,messages)
      results[counter].append(k)
    counter+=1
  import numpy as np
  triangle = np.triu (results)
  import pandas as pd
  return pd.DataFrame(triangle, columns=[brands], index=[brands])

In [ ]:
#made array with all the messsages with brand 
messages_with_brands = []
for message in tqdm(sentence_with_stopwords):
  tokenizer = nltk.RegexpTokenizer(r"\w+")
  message_tok = tokenizer.tokenize(message)
  words = " ".join(message_tok).lower().split()
  messages_with_brands.append(" ".join(brand_changer(words)))

100%|██████████| 4987/4987 [00:08<00:00, 556.87it/s]


In [ ]:
lift_matrix = lift_everything(brands,messages_with_brands)
lift_matrix_full = np.maximum( lift_matrix, lift_matrix.transpose())
lift_matrix_full

,bmw,acura,audi,honda,infiniti,nissan,toyota,subaru,volkswagen,ford,volvo
bmw,1.0000,1.5674,1.7931,1.2278,2.0549,1.2399,1.4161,1.1969,1.5152,1.5193,1.5967
acura,1.5674,1.0000,2.0910,2.7287,3.0591,2.2153,1.8066,2.0618,1.8302,1.6380,3.2192
audi,1.7931,2.0910,1.0000,1.2702,2.5262,1.3756,1.1424,1.8478,4.2529,1.7349,4.0942
honda,1.2278,2.7287,1.2702,1.0000,1.6168,4.0245,4.0897,1.5716,2.5015,2.2897,2.2815
infiniti,2.0549,3.0591,2.5262,1.6168,1.0000,2.9827,1.2247,1.7757,2.0895,1.7193,2.6698
nissan,1.2399,2.2153,1.3756,4.0245,2.9827,1.0000,4.0740,1.6023,2.6878,1.7193,3.5562
toyota,1.4161,1.8066,1.1424,4.0897,1.2247,4.0740,1.0000,1.4593,2.3324,2.0912,2.4250
subaru,1.1969,2.0618,1.8478,1.5716,1.7757,1.6023,1.4593,1.0000,2.4899,1.7219,4.5417
volkswagen,1.5152,1.8302,4.2529,2.5015,2.0895,2.6878,2.3324,2.4899,1.0000,2.2146,3.1525
ford,1.5193,1.6380,1.7349,2.2897,1.7193,1.7193,2.0912,1.7219,2.2146,1.0000,3.6549
